## Umbralización para obtener detección con segmentación

In [1]:
!pip3 install ultralytics

# Análisis de Movimiento en Video con Segmentación y Mapas de Calor

Este proyecto implementa una técnica para analizar el movimiento en secuencias de video, combinando sustracción de fondo y segmentación de instancias mediante el modelo YOLOv11. El objetivo es identificar y rastrear objetos en movimiento, específicamente personas, y visualizar su trayectoria a través de un mapa de calor acumulativo.

## Descripción del Código

1. **Inicialización**: Se carga el modelo YOLOv11 para segmentación (`yolo11n-seg`) y se inicializa el acumulador del mapa de calor `heatmap_refined`.

2. **Captura de Video**: Se procesa cada fotograma del video en un bucle continuo. Si no se puede leer un fotograma, el proceso se detiene.

3. **Sustracción de Fondo**: Se aplica un sustractor de fondo para obtener una máscara binaria que resalta las áreas en movimiento en el fotograma actual.

4. **Segmentación con YOLOv11**: Se utiliza el modelo YOLOv11 para realizar la segmentación de instancias en el fotograma, identificando objetos y generando máscaras para cada instancia detectada.

5. **Filtrado de Personas**: Se filtran las máscaras correspondientes a la clase "persona" (clase 0 en COCO) y se combinan en una única máscara de segmentación.

6. **Combinación de Máscaras**: Se realiza una intersección entre la máscara de movimiento y la máscara de segmentación de personas para obtener una máscara refinada que representa las áreas donde hay movimiento de personas.

7. **Actualización del Mapa de Calor**: La máscara refinada se acumula en el `heatmap_refined`, que visualiza las áreas con mayor actividad a lo largo del tiempo.

8. **Visualización**: Se muestran en ventanas separadas el fotograma original, la máscara de movimiento y la máscara refinada. El proceso continúa hasta que se presiona la tecla 'q'.

## Contexto Teórico

La combinación de sustracción de fondo y segmentación de instancias es una técnica poderosa en visión por computadora para detectar y rastrear objetos en movimiento. La sustracción de fondo permite identificar áreas de cambio en una escena estática, mientras que la segmentación de instancias, como la proporcionada por YOLOv11, permite identificar y delinear objetos específicos dentro de esas áreas. Al combinar ambas técnicas, es posible aislar y rastrear objetos de interés, como personas, en secuencias de video, facilitando aplicaciones en seguridad, análisis de comportamiento y más.


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

In [3]:
#video_path = "park_detection.avi"
video_path = "videos/people-detection.mp4"
models = "yolo11n-seg.pt"

In [4]:
# Abrir el video
cap = cv2.VideoCapture(video_path)

# Crear sustractor de fondo
bg_subtractor = cv2.createBackgroundSubtractorMOG2(
    history=500,          # Número de frames usados para construir el fondo.
    varThreshold=16,      # Sensibilidad para detectar cambios
    detectShadows=True,   # Detección de sombras
    )

In [5]:
heatmap_refined = None

# Cargar el modelo YOLOv11 para segmentación
model = YOLO("yolo11n-seg")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Inicializar el acumulador del heatmap en el primer frame
    if heatmap_refined is None:
        heatmap_refined = np.zeros(frame.shape[:2], dtype=np.float32)

    # --- Paso 1: Sustracción de Fondo ---
    fgmask = bg_subtractor.apply(frame)
    # Umbral para obtener una máscara binaria limpia
    _, fgmask = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)

    # --- Paso 2: Segmentación con YOLO ---
    # Realizamos la detección con segmentación sobre el frame completo.
    results = model(frame, verbose=False)[0]

    # Crear una máscara vacía para acumular las segmentaciones de la clase "persona"
    segmentation_mask = np.zeros(frame.shape[:2], dtype=np.uint8)

    if results.masks is not None:
        # Extraer las máscaras y las clases
        masks = results.masks.data.cpu().numpy() if hasattr(results.masks.data, 'cpu') else results.masks.data
        classes = results.boxes.cls.cpu().numpy() if hasattr(results.boxes.cls, 'cpu') else results.boxes.cls

        for mask, cls in zip(masks, classes):
            if int(cls) == 0:  # Filtramos detecciones de persona (en COCO, "person" es la clase 0)
                mask_bin = (mask > 0.5).astype(np.uint8) * 255
                # Redimensionar mask_bin a las dimensiones del frame (o segmentation_mask)
                mask_bin_resized = cv2.resize(mask_bin, (segmentation_mask.shape[1], segmentation_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
                segmentation_mask = cv2.bitwise_or(segmentation_mask, mask_bin_resized)

    # --- Paso 3: Combinación de Máscaras ---
    # Se realiza una intersección entre la máscara de movimiento y la máscara de segmentación de personas
    refined_mask = cv2.bitwise_and(fgmask, segmentation_mask)

    # Acumulamos la máscara refinada en el heatmap
    heatmap_refined = cv2.add(heatmap_refined, refined_mask.astype(np.float32))

    # Visualización intermedia
    cv2.imshow("Frame Original", frame)
    cv2.imshow("Mascara Movimiento (FG)", fgmask)
    # cv2.imshow("Mascara Segmentacion (Personas)", segmentation_mask)
    cv2.imshow("Mascara Refinada", refined_mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()